# 02 — Entraînement des modèles multiclasse

Entraîne 10 classifiers sur `HomeWin / Draw / AwayWin` avec split temporel (train < 2022, test ≥ 2022).

**Pré-requis :** avoir généré `data/all_leagues_combined.csv` avec le notebook `01_Data_Processing.ipynb`.

---
### Google Colab : configuration du chemin Drive

In [ ]:
# ============================================================
# CONFIGURATION — À modifier si nécessaire
# ============================================================
DRIVE_PROJECT_PATH = '/content/drive/MyDrive/FootWork'
LOCAL_PROJECT_PATH = '..'

In [ ]:
# ============================================================
# Détection environnement + montage Drive
# ============================================================
import os, sys

try:
    import google.colab
    ON_COLAB = True
except ImportError:
    ON_COLAB = False

if ON_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    PROJECT_ROOT = DRIVE_PROJECT_PATH
else:
    PROJECT_ROOT = os.path.abspath(LOCAL_PROJECT_PATH)

os.chdir(PROJECT_ROOT)
sys.path.insert(0, PROJECT_ROOT)
print(f'Environnement : {"Google Colab" if ON_COLAB else "Local"}')
print(f'Répertoire de travail : {os.getcwd()}')

In [ ]:
# ============================================================
# Installation des dépendances
# ============================================================
if ON_COLAB:
    %pip install -q pyyaml xgboost lightgbm catboost scikit-learn seaborn tqdm

In [ ]:
# ============================================================
# Imports
# ============================================================
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from tqdm.auto import tqdm

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, label_binarize
from sklearn.metrics import (confusion_matrix, classification_report, accuracy_score,
                              balanced_accuracy_score, matthews_corrcoef, cohen_kappa_score,
                              roc_curve, auc)
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.calibration import CalibratedClassifierCV, CalibrationDisplay
from sklearn.frozen import FrozenEstimator
from sklearn.ensemble import (RandomForestClassifier, GradientBoostingClassifier,
                               AdaBoostClassifier, ExtraTreesClassifier)
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
import lightgbm as lgb
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

from src.Config.Config_Manager import ConfigManager
from src.Models.threshold_optimizer import (
    find_optimal_thresholds_multiclass, predict_with_thresholds
)

config = ConfigManager('src/Config/configMC_1.yaml')
print('Config chargée.')

---
## Chargement et split temporel des données

In [ ]:
input_path = config.get_paths()['full_dataset']
exclude_columns = config.get_config_value('excluded_columns', default=[])
split_config = config.get_config_value('data_split', default={})

df = pd.read_csv(input_path)
print(f'Dataset chargé : {len(df)} matchs × {len(df.columns)} colonnes')
print(f'\nDistribution cible :')
print(df['target_result'].value_counts())

In [ ]:
# Split temporel :
#   train  : year < 2020
#   cal    : 2020 <= year < 2022  (calibration + optimisation des seuils)
#   test   : year >= 2022
temporal_split_year = split_config.get('temporal_split_year', 2022)
cal_split_year      = split_config.get('cal_split_year', 2020)

if 'date' in df.columns:
    df['date'] = pd.to_datetime(df['date'])
    train_df = df[df['date'].dt.year < cal_split_year].copy()
    cal_df   = df[(df['date'].dt.year >= cal_split_year) &
                  (df['date'].dt.year < temporal_split_year)].copy()
    test_df  = df[df['date'].dt.year >= temporal_split_year].copy()
    print(f'Split temporel :')
    print(f'  train  < {cal_split_year}        → {len(train_df)} matchs')
    print(f'  cal    {cal_split_year}–{temporal_split_year-1}      → {len(cal_df)} matchs')
    print(f'  test  >= {temporal_split_year}       → {len(test_df)} matchs')
else:
    random_params = {k: v for k, v in split_config.items() if k != 'temporal_split_year'}
    train_df, test_df = train_test_split(df, **random_params)
    cal_df = None
    print(f'Split aléatoire : train {len(train_df)} | test {len(test_df)}')

cols_to_drop = [c for c in exclude_columns if c in df.columns and c != 'target_result']
X_train = train_df.drop(cols_to_drop + ['target_result'], axis=1, errors='ignore')
X_test  = test_df.drop(cols_to_drop + ['target_result'], axis=1, errors='ignore')
X_cal   = cal_df.drop(cols_to_drop + ['target_result'], axis=1, errors='ignore') if cal_df is not None else None

le = LabelEncoder()
le.fit(df['target_result'])
y_train = le.transform(train_df['target_result'])
y_test  = le.transform(test_df['target_result'])
y_cal   = le.transform(cal_df['target_result']) if cal_df is not None else None
class_names = le.classes_

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)
X_cal_scaled   = scaler.transform(X_cal) if X_cal is not None else None

print(f'\nFeatures utilisées : {X_train.shape[1]}')
print(f'Classes : {class_names}')

---
## Définition des modèles

In [ ]:
model_params = config.get_config_value('model_parameters', default={})

catboost_dir = Path('src/Models/Multiclass_Target/catboost_info')
catboost_dir.mkdir(parents=True, exist_ok=True)

models = {
    'Random Forest':      RandomForestClassifier(**model_params.get('random_forest', {})),
    'Logistic Regression': LogisticRegression(**model_params.get('logistic_regression', {})),
    'SVM':                SVC(**model_params.get('svm', {})),
    'Gradient Boosting':  GradientBoostingClassifier(**model_params.get('gradient_boosting', {})),
    'XGBoost':            XGBClassifier(**model_params.get('xgboost', {})),
    'LightGBM':           LGBMClassifier(**model_params.get('lightgbm', {})),
    'CatBoost':           CatBoostClassifier(**model_params.get('catboost', {}),
                              train_dir=str(catboost_dir)),
    'KNN':                KNeighborsClassifier(**model_params.get('knn', {})),
    'AdaBoost':           AdaBoostClassifier(**model_params.get('adaboost', {})),
    'Extra Trees':        ExtraTreesClassifier(**model_params.get('extra_trees', {})),
}
print(f'{len(models)} modèles prêts.')

In [ ]:
# ============================================================
# Helper : entraînement avec suivi des courbes de loss
# ============================================================
def _fit_with_tracking(model, X_train, X_test, y_train, y_test, sample_weight=None):
    """Entraîne et capture les métriques par itération pour les boosters.

    sample_weight est propagé aux modèles qui n'acceptent pas class_weight
    dans leur constructeur (XGBoost, GradientBoosting, AdaBoost).
    """
    cls = model.__class__.__name__

    if cls == 'XGBClassifier':
        model.fit(X_train, y_train,
                  sample_weight=sample_weight,
                  eval_set=[(X_train, y_train), (X_test, y_test)],
                  verbose=False)
        evals = model.evals_result()
        mk = list(evals['validation_0'].keys())[0]
        return model, {'train': evals['validation_0'][mk],
                       'val':   evals['validation_1'][mk], 'metric': mk}

    elif cls == 'LGBMClassifier':
        evals_result = {}
        model.fit(X_train, y_train,
                  eval_set=[(X_train, y_train), (X_test, y_test)],
                  eval_names=['train', 'val'],
                  callbacks=[lgb.record_evaluation(evals_result)])
        mk = list(evals_result['train'].keys())[0]
        return model, {'train': evals_result['train'][mk],
                       'val':   evals_result['val'][mk], 'metric': mk}

    elif cls == 'CatBoostClassifier':
        base_dir = model.get_param('train_dir') or 'catboost_info'
        safe_dir = os.path.join(base_dir, f'run_{id(model)}')
        os.makedirs(safe_dir, exist_ok=True)
        model.set_params(train_dir=safe_dir)
        model.fit(X_train, y_train, eval_set=(X_test, y_test))
        evals = model.get_evals_result()
        lk = 'learn'      if 'learn'      in evals else list(evals.keys())[0]
        vk = 'validation' if 'validation' in evals else list(evals.keys())[-1]
        mk = list(evals[lk].keys())[0]
        return model, {'train': evals[lk][mk], 'val': evals[vk][mk], 'metric': mk}

    elif cls == 'GradientBoostingClassifier':
        model.fit(X_train, y_train, sample_weight=sample_weight)
        t = [accuracy_score(y_train, p) for p in model.staged_predict(X_train)]
        v = [accuracy_score(y_test,  p) for p in model.staged_predict(X_test)]
        return model, {'train': t, 'val': v, 'metric': 'accuracy'}

    elif cls == 'AdaBoostClassifier':
        model.fit(X_train, y_train, sample_weight=sample_weight)
        return model, None

    else:
        model.fit(X_train, y_train)
        return model, None

print('_fit_with_tracking défini.')

In [ ]:
output_dir = Path(config.get_paths()['output_dir'])
output_dir.mkdir(parents=True, exist_ok=True)

# Sample weights for models without native class_weight support
sample_weight = compute_sample_weight('balanced', y_train)

results = {}

pbar = tqdm(models.items(), total=len(models), desc="Training models")
for name, model in pbar:
    pbar.set_description(f"Training  {name:<22}")
    try:
        model, loss_history = _fit_with_tracking(
            model, X_train_scaled, X_test_scaled, y_train, y_test,
            sample_weight=sample_weight
        )
        y_pred = model.predict(X_test_scaled)

        # ── Métriques ────────────────────────────────────────
        acc     = accuracy_score(y_test, y_pred)
        bal_acc = balanced_accuracy_score(y_test, y_pred)
        mcc     = matthews_corrcoef(y_test, y_pred)
        kappa   = cohen_kappa_score(y_test, y_pred)
        cm      = confusion_matrix(y_test, y_pred)
        cr      = classification_report(y_test, y_pred, target_names=class_names)
        pbar.set_postfix(acc=f"{acc:.4f}", mcc=f"{mcc:.4f}")

        # ── ROC multiclasse (OvR) ─────────────────────────────
        macro_auc, per_class_auc, probs = None, {}, None
        if hasattr(model, 'predict_proba'):
            try:
                probs = model.predict_proba(X_test_scaled)
                n = len(class_names)
                y_bin = label_binarize(y_test, classes=list(range(n)))
                fpr_d, tpr_d, auc_d = {}, {}, {}
                for i in range(n):
                    fpr_d[i], tpr_d[i], _ = roc_curve(y_bin[:, i], probs[:, i])
                    auc_d[i] = auc(fpr_d[i], tpr_d[i])
                per_class_auc = auc_d
                all_fpr  = np.unique(np.concatenate([fpr_d[i] for i in range(n)]))
                mean_tpr = np.zeros_like(all_fpr)
                for i in range(n):
                    mean_tpr += np.interp(all_fpr, fpr_d[i], tpr_d[i])
                mean_tpr /= n
                macro_auc = auc(all_fpr, mean_tpr)

                fig, ax = plt.subplots(figsize=(6, 5))
                colors = plt.cm.Set1(np.linspace(0, 0.8, n))
                for i, (cls_name, color) in enumerate(zip(class_names, colors)):
                    ax.plot(fpr_d[i], tpr_d[i], color=color, lw=2,
                            label=f'{cls_name} (AUC={auc_d[i]:.3f})')
                ax.plot(all_fpr, mean_tpr, 'k--', lw=2,
                        label=f'Macro avg (AUC={macro_auc:.3f})')
                ax.plot([0,1],[0,1],'gray',ls=':',lw=1)
                ax.set_title(f'ROC OvR — {name}')
                ax.set_xlabel('FPR'); ax.set_ylabel('TPR')
                ax.legend(loc='lower right', fontsize=8)
                fig.tight_layout()
                fig.savefig(output_dir / f'roc_{name.replace(" ","_")}.png', dpi=150)
                plt.show()
            except Exception as e:
                print(f'  [ROC skipped]: {e}')

        # ── Matrice de confusion ──────────────────────────────
        fig, ax = plt.subplots(figsize=(6, 5))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                    xticklabels=class_names, yticklabels=class_names, ax=ax)
        ax.set_title(f'{name}  Acc={acc:.3f}  MCC={mcc:.3f}')
        ax.set_ylabel('Vrai label'); ax.set_xlabel('Label prédit')
        fig.tight_layout()
        fig.savefig(output_dir / f'cm_{name.replace(" ","_")}.png', dpi=150)
        plt.show()

        # ── Courbe de loss (boosters uniquement) ──────────────
        if loss_history:
            lh = loss_history
            _LOSS_METRICS = {'loss', 'logloss', 'multiclass', 'merror', 'mae', 'mse', 'rmse'}
            is_loss = any(tok in lh['metric'].lower() for tok in _LOSS_METRICS)
            best = int(np.argmin(lh['val']) if is_loss else np.argmax(lh['val']))
            fig, ax = plt.subplots(figsize=(7, 4))
            iters = range(1, len(lh['train']) + 1)
            ax.plot(iters, lh['train'], label='Train')
            ax.plot(iters, lh['val'],   label='Validation')
            ax.axvline(best + 1, color='red', ls='--', alpha=0.6, label=f'Best={best+1}')
            ax.set_title(f'Training Curve — {name}  ({lh["metric"]})')
            ax.set_xlabel('Itération'); ax.set_ylabel(lh['metric'])
            ax.legend(); fig.tight_layout()
            fig.savefig(output_dir / f'loss_curve_{name.replace(" ","_")}.png', dpi=150)
            plt.show()

        # ── Calibration + seuils optimisés ───────────────────
        # Run sequentially here (no threading) so exceptions are visible.
        calibrated_model, calibrated_auc = None, None
        opt_thresholds, acc_opt, bal_acc_opt, report_opt = None, None, None, None

        if X_cal_scaled is not None and probs is not None:
            try:
                calibrated_model = CalibratedClassifierCV(FrozenEstimator(model), method='isotonic')
                calibrated_model.fit(X_cal_scaled, y_cal)

                cal_probs_test = calibrated_model.predict_proba(X_test_scaled)
                n = len(class_names)
                y_bin = label_binarize(y_test, classes=list(range(n)))
                all_fpr  = np.unique(np.concatenate([
                    roc_curve(y_bin[:, i], cal_probs_test[:, i])[0] for i in range(n)
                ]))
                mean_tpr = np.zeros_like(all_fpr)
                for i in range(n):
                    fpr_i, tpr_i, _ = roc_curve(y_bin[:, i], cal_probs_test[:, i])
                    mean_tpr += np.interp(all_fpr, fpr_i, tpr_i)
                mean_tpr /= n
                calibrated_auc = auc(all_fpr, mean_tpr)

                opt_thresholds = find_optimal_thresholds_multiclass(
                    calibrated_model, X_cal_scaled, y_cal
                )
                y_pred_opt = predict_with_thresholds(cal_probs_test, opt_thresholds)
                acc_opt     = accuracy_score(y_test, y_pred_opt)
                bal_acc_opt = balanced_accuracy_score(y_test, y_pred_opt)
                report_opt  = classification_report(y_test, y_pred_opt, target_names=class_names)

                print(f'  {name}: cal_auc={calibrated_auc:.4f}  acc_opt={acc_opt:.4f}')
            except Exception as e:
                import traceback
                print(f'  [Calibration FAILED for {name}]:')
                traceback.print_exc()

        results[name] = {
            'accuracy': acc, 'balanced_accuracy': bal_acc,
            'mcc': mcc, 'kappa': kappa,
            'macro_roc_auc': macro_auc, 'per_class_auc': per_class_auc,
            'loss_history': loss_history, 'confusion_matrix': cm, 'report': cr,
            'calibrated_model': calibrated_model,
            'calibrated_macro_auc': calibrated_auc,
            'optimal_thresholds': opt_thresholds,
            'accuracy_opt': acc_opt,
            'balanced_accuracy_opt': bal_acc_opt,
            'report_opt': report_opt,
        }

    except Exception as e:
        print(f'  ERREUR : {e}')
        import traceback; traceback.print_exc()

print('\n=== Entraînement terminé ===')

In [ ]:
# Tableau de classement complet
rows = []
for name, res in results.items():
    row = {
        'Modèle':    name,
        'Accuracy':  res['accuracy'],
        'Bal. Acc':  res['balanced_accuracy'],
        'MCC':       res['mcc'],
        'Kappa':     res['kappa'],
        'Macro AUC': res.get('macro_roc_auc') or 0.0,
    }
    for i, cls_name in enumerate(class_names):
        row[f'AUC {cls_name}'] = res['per_class_auc'].get(i, np.nan)
    rows.append(row)

summary = pd.DataFrame(rows).sort_values('Accuracy', ascending=False)
pd.options.display.float_format = '{:.4f}'.format
print(summary.to_string(index=False))

# Graphique comparatif
model_names = summary['Modèle'].tolist()
x, w = np.arange(len(model_names)), 0.35

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

ax = axes[0]
ax.bar(x - w/2, summary['Accuracy'],  w, label='Accuracy',     color='steelblue')
ax.bar(x + w/2, summary['Bal. Acc'],  w, label='Balanced Acc', color='coral')
ax.set_xticks(x); ax.set_xticklabels(model_names, rotation=30, ha='right')
ax.set_ylim([0, 1]); ax.set_ylabel('Score'); ax.legend()
ax.set_title('Accuracy vs Balanced Accuracy')

ax = axes[1]
ax.bar(x, summary['Macro AUC'], color='mediumseagreen')
ax.axhline(0.5, color='red', ls='--', alpha=0.5, label='Random')
ax.set_xticks(x); ax.set_xticklabels(model_names, rotation=30, ha='right')
ax.set_ylim([0, 1]); ax.set_ylabel('Macro AUC (OvR)'); ax.legend()
ax.set_title('Macro AUC par modèle')

fig.tight_layout()
fig.savefig(output_dir / 'summary_comparison.png', dpi=150)
plt.show()

In [ ]:
metrics_file = output_dir / 'metrics_results_Multiclass.txt'
sorted_res = sorted(results.items(), key=lambda x: x[1]['accuracy'], reverse=True)

with open(metrics_file, 'w') as f:
    f.write('Results — Multiclass Prediction (temporal split)\n')
    f.write('=' * 70 + '\n\n')

    # Ranking table
    f.write('=== RANKING TABLE ===\n')
    f.write(f"{'Modèle':<22} {'Accuracy':>9} {'Bal.Acc':>9} {'MCC':>8} {'Kappa':>8} {'MacroAUC':>10}\n")
    f.write('-' * 72 + '\n')
    for name, res in sorted_res:
        auc_str = f"{res['macro_roc_auc']:.4f}" if res.get('macro_roc_auc') else '  N/A  '
        f.write(f"{name:<22} {res['accuracy']:>9.4f} {res['balanced_accuracy']:>9.4f} "
                f"{res['mcc']:>8.4f} {res['kappa']:>8.4f} {auc_str:>10}\n")
    f.write('\n')

    # Per-class AUC
    f.write('=== PER-CLASS AUC (OvR) ===\n')
    f.write(f"{'Modèle':<22}" + ''.join(f'{c:>12}' for c in class_names) + '\n')
    f.write('-' * (22 + 12 * len(class_names)) + '\n')
    for name, res in sorted_res:
        row = f'{name:<22}'
        for i in range(len(class_names)):
            v = res['per_class_auc'].get(i)
            row += f'{v:>12.4f}' if v is not None else f"{'N/A':>12}"
        f.write(row + '\n')
    f.write('\n')

    # Calibrated vs Uncalibrated
    f.write('=== CALIBRATED vs UNCALIBRATED MACRO AUC ===\n')
    f.write(f"{'Modèle':<22} {'Uncalibrated':>14} {'Calibrated':>12} {'Delta':>8}\n")
    f.write('-' * 60 + '\n')
    for name, res in sorted_res:
        raw = res.get('macro_roc_auc')
        cal = res.get('calibrated_macro_auc')
        raw_s = f"{raw:.4f}" if raw is not None else "   N/A"
        cal_s = f"{cal:.4f}" if cal is not None else "   N/A"
        delta_s = f"{cal - raw:+.4f}" if (raw is not None and cal is not None) else "   N/A"
        f.write(f"{name:<22} {raw_s:>14} {cal_s:>12} {delta_s:>8}\n")
    f.write('\n')

    # Optimised threshold metrics
    f.write('=== OPTIMISED THRESHOLD METRICS ===\n')
    f.write(f"{'Modèle':<22} {'Acc(default)':>13} {'Acc(opt)':>10} {'BalAcc(default)':>16} {'BalAcc(opt)':>12}\n")
    f.write('-' * 77 + '\n')
    for name, res in sorted_res:
        acc_o_s = f"{res['accuracy_opt']:.4f}" if res.get('accuracy_opt') is not None else "   N/A"
        bal_o_s = f"{res['balanced_accuracy_opt']:.4f}" if res.get('balanced_accuracy_opt') is not None else "   N/A"
        f.write(f"{name:<22} {res['accuracy']:>13.4f} {acc_o_s:>10} "
                f"{res['balanced_accuracy']:>16.4f} {bal_o_s:>12}\n")
    f.write('\n')

    # Detailed
    f.write('=== DETAILED RESULTS ===\n\n')
    for name, res in sorted_res:
        f.write(f'Modèle : {name}\n')
        f.write('-' * 30 + '\n')
        f.write(f"Accuracy          : {res['accuracy']:.4f}\n")
        f.write(f"Balanced Accuracy : {res['balanced_accuracy']:.4f}\n")
        f.write(f"MCC               : {res['mcc']:.4f}\n")
        f.write(f"Cohen's Kappa     : {res['kappa']:.4f}\n")
        if res.get('macro_roc_auc'):
            f.write(f"Macro ROC-AUC     : {res['macro_roc_auc']:.4f}\n")
        if res.get('calibrated_macro_auc'):
            f.write(f"Calibrated AUC    : {res['calibrated_macro_auc']:.4f}\n")
        if res.get('accuracy_opt') is not None:
            f.write(f"Accuracy (opt)    : {res['accuracy_opt']:.4f}\n")
            f.write(f"Bal.Acc  (opt)    : {res['balanced_accuracy_opt']:.4f}\n")
        f.write('\nClassification Report :\n')
        f.write(res['report'])
        if res.get('report_opt'):
            f.write('\nClassification Report (Optimised Thresholds) :\n')
            f.write(res['report_opt'])
        if res.get('optimal_thresholds'):
            f.write('\nOptimal Thresholds :\n')
            for i, (t, f1) in res['optimal_thresholds'].items():
                f.write(f"  {class_names[i]}: threshold={t:.4f}, best_F1={f1:.4f}\n")
        f.write('\nConfusion Matrix :\n')
        f.write(str(res['confusion_matrix']))
        lh = res.get('loss_history')
        if lh:
            _LOSS_METRICS = {'loss', 'logloss', 'multiclass', 'merror', 'mae', 'mse', 'rmse'}
            is_loss = any(tok in lh['metric'].lower() for tok in _LOSS_METRICS)
            best = int(np.argmin(lh['val']) if is_loss else np.argmax(lh['val']))
            f.write(f"\nTraining curve: metric={lh['metric']}, best_iter={best+1}, "
                    f"final_train={lh['train'][-1]:.4f}, final_val={lh['val'][-1]:.4f}\n")
        f.write('\n' + '=' * 70 + '\n')

print(f'Métriques sauvegardées : {metrics_file}')